In [2]:
import magellan as mg
import pandas as pd

In [3]:
dblp = mg.read_csv('DBLP_cleaned.csv', key='id')
acm = mg.read_csv('ACM_cleaned.csv', key='id')

In [4]:
ab = mg.AttrEquivalenceBlocker()
c = ab.block_tables(dblp, acm, 'year', 'year', ['title', 'authors', 'year', 'venue'],
                   ['title', 'authors', 'year', 'venue'])

In [5]:
gold = mg.read_csv('dblp-acm_gold.csv', key='pairId')
gold.columns = ['id', 'ltable.id', 'rtable.id', 'label']
g = mg.create_mtable(gold, key='id', ltable=dblp, rtable=acm, 
                     foreign_key_ltable='ltable.id', 
                     foreign_key_rtable='rtable.id')
g.set_index(['ltable.id', 'rtable.id'], inplace=True, drop=False)

In [22]:
ob = mg.OverlapBlocker()
d = ob.block_tables(dblp, acm, 'title', 'title', l_output_attrs=['title', 'authors'], 
                    r_output_attrs=['title', 'authors'],
                    overlap_size=3)

0%                          100%
[##############################] | ETA[sec]: 0.000 
Total time elapsed: 7.046 sec


In [23]:
e = ob.block_candset(d, 'authors', 'authors', overlap_size=1)

0%                          100%
[##############################] | ETA[sec]: 0.000 
Total time elapsed: 4.242 sec


In [24]:
len(e)

6799

In [28]:
f = ob.block_tables(dblp, acm, 'authors', 'authors', l_output_attrs=['title', 'authors'], 
                    r_output_attrs=['title', 'authors'],
                    overlap_size=2)

0%                          100%
[##############################] | ETA[sec]: 0.000 
Total time elapsed: 6.170 sec


In [29]:
len(f)

60627

In [30]:
f.head()

,_id,ltable.id,rtable.id,ltable.title,ltable.authors,rtable.title,rtable.authors
0,0,conf/sigmod/AbadiC02,191915,Visual COKO: a debugger for query optimizer de...,"Daniel J. Abadi, Mitch Cherniack",Shoring up persistent applications,"Michael J. Carey, David J. DeWitt, Michael J. ..."
1,1,conf/sigmod/AbadiC02,191931,Visual COKO: a debugger for query optimizer de...,"Daniel J. Abadi, Mitch Cherniack",DEC data distributor: for data replication and...,Daniel J. Dietterich
2,2,conf/sigmod/AbadiC02,233356,Visual COKO: a debugger for query optimizer de...,"Daniel J. Abadi, Mitch Cherniack",Rule languages and internal algebras for rule-...,"Mitch Cherniack, Stanley B. Zdonik"
3,3,conf/sigmod/AbadiC02,276311,Visual COKO: a debugger for query optimizer de...,"Daniel J. Abadi, Mitch Cherniack",Changing the rules: transformations for rule-b...,"Mitch Cherniack, Stan Zdonik"
4,4,conf/sigmod/AbadiC02,564770,Visual COKO: a debugger for query optimizer de...,"Daniel J. Abadi, Mitch Cherniack",Visual COKO: a debugger for query optimizer de...,"Daniel J. Abadi, Mitch Cherniack"


In [31]:
s = mg.sample_table(c, 50)
s.set_index(['ltable.id', 'rtable.id'], inplace=True, drop=False)


In [42]:
se = mg.sample_table(e, 120)
se.set_index(['ltable.id', 'rtable.id'], inplace=True, drop=False)

In [34]:
sf = mg.sample_table(f, 50)
sf.set_index(['ltable.id', 'rtable.id'], inplace=True, drop=False)

In [35]:
g1 = mg.sample_table(g, 200)


In [43]:
s1 = set(s.index.values)
s2 = set(se.index.values)
s3 = set(sf.index.values)
s4 = set(g1.index.values)

In [44]:
s5 = s1.union(s2).union(s3)

In [45]:
s5.intersection(s4)

{('conf/vldb/KahveciS01', 672028)}

In [46]:
g1.reset_index(inplace=True, drop=True)
s.reset_index(inplace=True, drop=True)
se.reset_index(inplace=True, drop=True)

sf.reset_index(inplace=True, drop=True)




In [47]:
l = mg.combine_block_outputs_via_union([g1, s, se, sf])

In [48]:
len(l)

419

In [49]:
l['gold'] = 0

In [50]:
g1.head()

,id,ltable.id,rtable.id,label
0,30,conf/sigmod/ButtlerLPP01,375762,1
1,102,conf/sigmod/WaasG00,335451,1
2,135,conf/sigmod/ChangBCLLS00,335433,1
3,138,conf/sigmod/RaoR00,335449,1
4,146,conf/sigmod/Gal00,335479,1


In [51]:
l['gold'] = 0

In [52]:
l.set_index(['ltable.id', 'rtable.id'], inplace=True, drop=False)

In [53]:
mg.view(l)

In [54]:
g1.set_index(['ltable.id', 'rtable.id'], inplace=True, drop=False)

In [55]:
m = set(g1.index.values)
n = set(l.index.values)

In [19]:
l.head()

,,_id,ltable.id,rtable.id,ltable.authors,ltable.title,ltable.venue,ltable.year,rtable.authors,rtable.title,rtable.venue,rtable.year,gold
ltable.id,rtable.id,,,,,,,,,,,,
conf/sigmod/AcharyaAFZ95,673141,0,conf/sigmod/AcharyaAFZ95,673141,"Michael J. Franklin, Stanley B. Zdonik, Swarup...",Broadcast Disks: Data Management for Asymmetri...,SIGMOD Conference,1995,"Kazutoshi Sumiya, Kouichi Yasutake, Hirohiko T...",A Product Specification Database for Visual Pr...,Very Large Data Bases,1995,0
conf/sigmod/AcharyaGPR99,304589,1,conf/sigmod/AcharyaGPR99,304589,"Sridhar Ramaswamy, Swarup Acharya, Phillip B. ...",The Aqua Approximate Query Answering System,SIGMOD Conference,1999,"Athman Bouguettaya, Boualem Benatallah, Lily H...","World Wide Database-integrating the Web, CORBA...",International Conference on Management of Data,1999,0
conf/sigmod/AdaliCPS96,233359,2,conf/sigmod/AdaliCPS96,233359,"Sibel Adali, Yannis Papakonstantinou, K. Seluk...",Query Caching and Optimization in Distributed ...,SIGMOD Conference,1996,"Joseph M. Hellerstein, Jeffrey F. Naughton",Query execution techniques for caching expensi...,International Conference on Management of Data,1996,0
conf/sigmod/AggarwalPWYP99,304188,3,conf/sigmod/AggarwalPWYP99,304188,"Joel L. Wolf, Cecilia Magdalena Procopiuc, Jon...",Fast Algorithms for Projected Clustering,SIGMOD Conference,1999,"Charu C. Aggarwal, Joel L. Wolf, Philip S. Yu,...",Fast algorithms for projected clustering,International Conference on Management of Data,1999,0
conf/sigmod/AltinelABFSZ99,758373,4,conf/sigmod/AltinelABFSZ99,758373,"Demet Aksoy, Mehmet Altinel, Stanley B. Zdonik...",DBIS-Toolkit: Adaptable Middleware for Large S...,SIGMOD Conference,1999,"H. V. Jagadish, Nick Koudas, S. Muthukrishnan",Mining Deviants in a Time Series Database,Very Large Data Bases,1999,0


In [56]:
l.ix[g1.index.values, 'gold'] = 1

In [61]:
mg.edit(l)

/scratch/pradap/local/share/anaconda/lib/python2.7/site-packages/Magellan-0.1-py2.7.egg/magellan/gui/mtable_gui.py:58: FutureWarning: iget_value(i, j) is deprecated. Please use .iat[i, j]
  inp = tbl.iget_value(i, j)


In [60]:
len(m.intersection(n))

200

In [23]:
l.reset_index(drop=True, inplace=True)

In [55]:
l.drop(282, inplace=True)

In [56]:
mg.view(l)

In [62]:
l.to_csv('acm_dblp_label_1.csv')

True